In [1]:
import os
from pathlib import Path

import tensorflow as tf
import tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import StatisticsGen, CsvExampleGen

2023-01-10 02:00:11.232648: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# 컨텍스트 객체는 컴포넌트의 실행을 처리, 컴포넌트의 아티팩트를 표시한다.
# InteractiveContext 객체는 in-memory 메타데이터스토어도 설정한다.
context = InteractiveContext()

In [3]:
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
dir_path = Path().parent.absolute()
print(dir_path)
data_dir = os.path.join(dir_path, "data", "taxi")
Path(data_dir).mkdir(parents=True, exist_ok=True)
filepath = tf.keras.utils.get_file(
    os.path.join(data_dir, "data.csv"),
    DATA_PATH
)

/home/hakjun/projects/pipeline


In [4]:
# 컴포넌트는 입력 아티팩트의 메타데이터를 메타데이터스토어에서 받아 오고,
# 실행 후 출력 아티팩트 메타데이터를 메타데이터스토어에 저장한다.
# driver: 입력 메타데이터 저장, executor: 입력 아티팩트를 처리, publisher: 출력 메타데이터 저장
example_gen = CsvExampleGen(input_base=data_dir)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [11]:
# 컨텍스트 객체로 아티팩트를 검사할 수 있다.
context.show(example_gen)

CsvExampleGen(spec: <tfx.types.standard_component_specs.FileBasedExampleGenSpec object at 0x7fbd0a849700>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x7fbd0a849520>, driver_class: <class 'tfx.components.example_gen.driver.FileBasedDriver'>, component_id: CsvExampleGen, inputs: {}, outputs: {'examples': OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={})})

In [8]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples']
)
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [9]:
context.show(statistics_gen.outputs['statistics'])

In [12]:
for artifact in statistics_gen.outputs['statistics'].get():
    print(artifact)

Artifact(artifact: id: 2
type_id: 16
uri: "/tmp/tfx-interactive-2023-01-10T02_00_29.655379-dl52pq0f/StatisticsGen/statistics/2"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "statistics:2023-01-10T02:06:00.272120"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "StatisticsGen"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.12.0"
  }
}
state: LIVE
name: "statistics:2023-01-10T02:06:00.272120"
, artifact_type: id: 16
name: "ExampleStatistics"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
base_type: STATISTICS
)
